# Web Scraping - Chipotle

In [1]:
## Import all the libraries required

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from openpyxl import Workbook,load_workbook

#### Step 1:  Specify the route or the website 

In [2]:
# To Specify the route or website 
def url_fetcher(session, route, base_url='https://www.yelp.com', params=None):
    #params = kwargs if kwargs else None  ## checks whether the kwargs is empty or not. 
    
    if 'yelp' not in route:
        return session.get(base_url + route, params=params) 
        ## session.get() is used top send a GET request to the specified URL
    else:
        return session.get(route, params=params)

#### Step 2: Import the file with Zip Code

In [3]:
# Import the file with Zip Codes
zip_file=pd.read_excel('USA Zip Code.xlsx',sheet_name='Sheet5')

#### Step 3: Function to scrap the followig details of the restaurant 
1. Name of the resturant
2. Address
3. Rating
4. Review Count
5. Overall Feedback

In [4]:
def yelp_scraper(location, num=10):
    session = requests.Session()  ## to create a session object
    session.headers = {'User-Agent': 'Mozilla/5.0'}
    params = {'find_desc': 'Chipotle', 'find_loc': location, 'start': 0}
    response = url_fetcher(session, '/search', params=params)
    #while True:
    soup = BeautifulSoup(response.content, 'lxml') 
    for items in soup.findAll("div", attrs={"class" : "padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border-color--default__09f24__NPAKY"}):
        try:
            
            name=items.find("a", attrs={'class' :'css-1m051bw'}).get_text(strip=True)
        except Exception as e:
            name='Null'
        try:
            address = items.find("span", class_="raw__09f24__T4Ezm").text.strip()
        except Exception as e:
            address = 'Unknown'
        try:
            rating = items.find("div", attrs = {"class" : re.compile(r'five-stars_+')}).get('aria-label').split()[0]
        except Exception as e:
            rating = '0'
        try:
            review_count = items.findAll("span", attrs = {"class" : "css-chan6m"})[0].get_text(strip=True)
        except Exception as e:
            review_count = '0'
        try:
            overall_feedback = items.find("p", attrs = {"class" : "css-16lklrv"}).get_text(strip=True).split("...")[0]
        except Exception as e:
            #print(e)
            overall_feedback = 'None'
        yield name,location, address, rating, review_count, overall_feedback
    

#### Step 4: To convert all the Zip Codes as a list

In [5]:
zip_list = zip_file['zip Code'].to_list()

#### Step 5: To save as a data frame for all the list of Zip Codes

In [6]:
columns=['Resturant Name','Location','Address','Rating','Review Count','Overall Feedback']
final_dataframe=pd.DataFrame(columns=columns)

if __name__ == '__main__':
    for zip in zip_list:
        for name,location, address, rating, review_count, overall_feedback in yelp_scraper(zip):
                    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[name,location,address,rating,review_count,overall_feedback]], columns=columns)], ignore_index=True)
                    #print('Name:', name)
                    #print('Location:', location)
                    #print('Address:', address)
                    #print('Rating:', rating)
                    #print('Review count:', review_count)
                    #print('Overall feedback:', overall_feedback)
                    print(final_dataframe)

           Resturant Name Location              Address Rating Review Count  \
0  Chipotle Mexican Grill    44241  9754 State Route 14    3.5           45   

                                    Overall Feedback  
0  “Chipotle is a breath of fresh air after road ...  
           Resturant Name Location              Address Rating Review Count  \
0  Chipotle Mexican Grill    44241  9754 State Route 14    3.5           45   
1  Chipotle Mexican Grill    44241     5 Atterbury Blvd    2.5           42   

                                    Overall Feedback  
0  “Chipotle is a breath of fresh air after road ...  
1  “I have never had any issues dining in! Food i...  
           Resturant Name Location              Address Rating Review Count  \
0  Chipotle Mexican Grill    44241  9754 State Route 14    3.5           45   
1  Chipotle Mexican Grill    44241     5 Atterbury Blvd    2.5           42   
2  Chipotle Mexican Grill    44241        429 E Main St    2.5           49   

           

#### Step 6: To save the file in the desired location

In [7]:
# specify the folder path where you want to save the file
folder_path = '/Users/ujjaini.dalal/Desktop/Chipotle/Web Scrapping/'

# specify the file name and extension
file_name = 'Web Scraping Output.csv'

# save the DataFrame to the specified folder
final_dataframe.to_csv(folder_path + file_name, index=False)